# Sujet

L’objectif du projet est de concevoir un programme créant un portefeuille optimisant le ratio de Sharpe, sous certaines contraintes, comme le ferait un gestionnaire d’actif ou un robo-advisor.
L’univers d’actifs de base étant possiblement très grand (environs 450 actifs), le nombre de possibilité de compositions différentes envisageables est alors infini, ce qui rend ce problème NP-complet.
Votre objectif est de créer un unique portefeuille avec les contraintes suivantes :

-	Le portefeuille doit être exactement composé d’un minimum de 15 actifs, et d’un maximum de 40 actifs.
-	Chaque actif doit représenter un %NAV du portefeuille entre 1 et 10% à la date du 01/06/2016.
-	Le portefeuille n’aura qu’une unique composition commençant le 01/06/2016.
-	On évaluera le sharpe sur la période du 01/06/2016 au 30/09/2020
-	Le portefeuille devra comporter au moins 50% d’actions. (en montant pas en quantité)

Si l’une de ces contraintes n’est pas respectée votre portefeuille sera considéré comme invalide et non pris en compte dans le classement (et/ou Notation)

La valeur du ratio de Sharpe prise en compte dans la notation est celle de la période du 01/06/2016 au 30/09/2020. Les données mises à dispositions elles, s’arrêtent au 23/10/2020.


# Setup

## Install dependencies

In [1]:
!pip install requests python-dotenv

## Load environment variables

have a _.env_ file containing:
```
DOLPHIN_USERNAME=EPITA_GROUPE5
DOLPHIN_PASSWORD=pwd
```

In [2]:
# Date
from datetime import datetime

# Utils
import warnings

# Environment
from dotenv import load_dotenv
import os
from os import getenv
if not os.path.isfile('.env'):
  username = ''
  password = ''
  warnings.warn('.env file not present ! creating...')
  while not username or not password:
    username = input('DOLPHIN_USERNAME')
    password = input('DOLPHIN_PASSWORD')
  with open('.env', 'w') as f:
    f.write(f'DOLPHIN_USERNAME={username}\n')
    f.write(f'DOLPHIN_PASSWORD={password}')
# Load the .env environment variables
load_dotenv()

# Network
import urllib3
urllib3.disable_warnings()
from base64 import b64encode

# Requests
import requests
from requests.compat import urljoin
from requests import Session

# Dataframe
import numpy as np
import pandas as pd
import math

# Network classes

In [3]:
class NetworkManager():
    '''
    Low level network manager class; python requests module wrapper
    '''
    def __init__(self):
        # Username and Password
        self.__USERNAME = getenv('DOLPHIN_USERNAME')
        self.__PASSWORD = getenv('DOLPHIN_PASSWORD')
        if not self.__USERNAME or not self.__PASSWORD:
          raise RuntimeError('Variable DOLPHIN_USERNAME or DOLPHIN_PASSWORD is not set')
        self.__USERNAME = bytes(self.__USERNAME, 'utf-8')
        self.__PASSWORD = bytes(self.__PASSWORD, 'utf-8')
        
        # root
        self.SCHEME = 'https'
        self.HOST_NAME = 'dolphin.jump-technology.com'
        self.PORT = 8443
        self.APIVERSION = 'v1'
        self.URL = f'{self.SCHEME}://{self.HOST_NAME}:{self.PORT}/api/{self.APIVERSION}/'
        #self.HEADERS = { 'Authorization': self.__getAuth() }
        #self.session = Session(auth=(self.__USERNAME,self.__PASSWORD), headers=self.HEADERS)
        
        # session
        self.session = Session()
        self.session.auth = (self.__USERNAME, self.__PASSWORD)
        self.session.verify = False
    
    def __call__(self, endpoint, params):
        url = urljoin(self.URL, endpoint)
        return self.session.get(url, params=params)

    def post(self, endpoint, params, payload):
        url = urljoin(self.URL, endpoint)
        return self.session.post(url, params=params, json=payload)

    def put(self, endpoint, params, payload):
        url = urljoin(self.URL, endpoint)
        return self.session.put(url, params=params, json=payload)

    '''
    def __getAuth(self):
        return 'Basic ' + b64encode(b':'.join((self.__USERNAME, self.__PASSWORD))).strip().decode('utf-8')
    '''

In [4]:
class RestManager(NetworkManager):
    '''
    High level RestManager class exposing main Dolphin API methods
    '''
    def __init__(self):
        super(RestManager, self).__init__()
        
    def getAssetList(self, parDate, columns=['ASSET_DATABASE_ID','LABEL','LAST_CLOSE_VALUE_IN_CURR','TYPE','CURRENCY'], fullResponse=False):
        '''
        Retourne la liste complète des actifs.
        Avec les valeurs des colonnes "ASSET_DATABASE_ID", "LABEL", "LABEL_CLOSE_VALUE_IN_CURR", "TYPE", "CURRENCY" à parDate.

                Parameters:
                        parDate (str): an RFC 3339 compliant date string
                        columns (list): the list of parameters
                        fullResponse (bool): wether to return the full response or not

                Returns:
                        json (list): list of pars dictionary
        '''
        try:
            params = { 'date': parDate, 'columns': columns }
            if fullResponse:
              params['fullResponse'] = fullResponse
            resp = self('asset', params=params)
            return resp.json()
        except Exception as e:
            return e
    
    def getAsset(self, parId, parDate, columns=['ASSET_DATABASE_ID', 'LABEL', 'LAST_CLOSE_VALUE_IN_CURR', 'TYPE', 'CURRENCY'], fullResponse=False):
      try:
        params = {'date': parDate, 'columns': columns}
        if fullResponse:
          params['fullResponse'] = fullResponse
        resp = self(f'asset/{parId}', params=params)
        return resp.json()
      except Exception as e:
        return e

    def getAssetAttr(self, parId, parAttr, parDate, fullResponse=False):
      try:
        params = {'date': parDate}
        if fullResponse:
          params['fullResponse'] = fullResponse
        resp = self(f'asset/{parId}/attribute/{parAttr}', params=params)
        return resp.json()
      except Exception as e:
        return e

    def getQuote(self, parId, parStartDate, parEndDate):
      try:
        params = {'start_date': parStartDate, 'end_date': parEndDate}
        resp = self(f'asset/{parId}/quote', params=params)
        return resp.json()
      except Exception as e:
        return e

    def getPortfolio(self, parId):
      try:
        params = []
        resp = self(f'portfolio/{parId}/dyn_amount_compo', params=params)
        return resp.json()
      except Exception as e:
        return e

    def putPortfolio(self, parId, portfolioLabel, currency, amount_type, values):
      try:
        params = []
        payload = {'label': portfolioLabel,
                   'currency': currency,
                   'type': amount_type,
                   'values': values}
        resp = self.put(f'portfolio/{parId}/dyn_amount_compo', params=params, payload=payload)
        return resp.json()
      except Exception as e:
        return e
    
    def getRatio(self):
      try:
        params = []
        resp = self('ratio', params=params)
        return resp.json()
      except Exception as e:
        return e

    def putRatio(self, ratio, asset, benchmark, start_date, end_date, frequency, fullResponse=False):
      try:
        params = []
        if fullResponse:
          params['fullResponse'] = fullResponse
        payload = {'ratio': ratio,
                   'asset': asset,
                   'benchmark': benchmark,
                   'start_date': start_date,
                   'end_date': end_date,
                   'frequency': frequency}
        resp = self.post('ratio/invoke', params=params, payload=payload)
        return resp.json()
      except Exception as e:
        return e

## Examples

In [5]:
r = RestManager()

In [6]:
ratios = r.getRatio()

In [7]:
ratios = pd.DataFrame.from_dict(ratios).set_index("id")
ratios

,type,name,is_benchmark_needed,is_percent
id,,,,
11,Ratio,Correlation,True,False
7,Ratio,Exposition action,False,True
15,Ratio,Exposition action,False,True
8,Ratio,RDT,False,True
13,Ratio,Rendement,False,True
9,Ratio,Rendement_annualise,False,True
12,Ratio,Sharpe,False,False
14,Ratio,VaR historique,False,True
10,Ratio,Volatilite,False,True


# Portfolio creation

### AssetList

In [8]:
pars = r.getAssetList(datetime(2016, 6, 1).isoformat('T'))

In [9]:
pars

[{'CURRENCY': {'type': 'asset_currency', 'value': 'EUR'},
  'ASSET_DATABASE_ID': {'type': 'int32', 'value': '1845'},
  'TYPE': {'type': 'asset_type', 'value': 'STOCK'},
  'LAST_CLOSE_VALUE_IN_CURR': {'type': 'currency_value',
   'value': '36,851 EUR'},
  'LABEL': {'type': 'string', 'value': '1&1 DRILLISCH AG'}},
 {'CURRENCY': {'type': 'asset_currency', 'value': 'EUR'},
  'ASSET_DATABASE_ID': {'type': 'int32', 'value': '1846'},
  'TYPE': {'type': 'asset_type', 'value': 'STOCK'},
  'LAST_CLOSE_VALUE_IN_CURR': {'type': 'currency_value', 'value': '40,9 EUR'},
  'LABEL': {'type': 'string', 'value': '1000MERCIS'}},
 {'CURRENCY': {'type': 'asset_currency', 'value': 'EUR'},
  'ASSET_DATABASE_ID': {'type': 'int32', 'value': '2122'},
  'TYPE': {'type': 'asset_type', 'value': 'FUND'},
  'LAST_CLOSE_VALUE_IN_CURR': {'type': 'currency_value', 'value': '39,37 EUR'},
  'LABEL': {'type': 'string', 'value': '1818-VEGA EUR CO'}},
 {'CURRENCY': {'type': 'asset_currency', 'value': 'EUR'},
  'ASSET_DATABAS

### Asset list dataframe

Let's first create a dataframe that regroups all the information we could get beforehand from the API.

In [10]:
START_DATE = datetime(2016, 6, 1).isoformat('T')
END_DATE = datetime(2020, 9, 30).isoformat('T')
MIN_PERCENT_NAV = 0.01
MAX_PERCENT_NAX = 0.1
NB_MIN_ASSET = 15
NB_MAX_ASSET = 40
PERCENT_MIN_STOCK = 0.5

In [156]:
class Portfolio():
    
    def __init__(self, dataframe=None): 
        if dataframe is None :
            pars = r.getAssetList(datetime(2016, 6, 1).isoformat('T'))
            company_id_array = []
            company_name_array = []
            asset_currency_array = []
            asset_type_array = []
            asset_value_array = []

            for asset in pars:
                company_id_array.append(int(asset['ASSET_DATABASE_ID']['value']))
                company_name_array.append(asset['LABEL']['value'])
                asset_currency_array.append(asset['CURRENCY']['value'])
                asset_type_array.append(asset['TYPE']['value'])
                asset_value_array.append(asset['LAST_CLOSE_VALUE_IN_CURR']['value'].split()[0].replace(',', '.')) if 'LAST_CLOSE_VALUE_IN_CURR' in asset else asset_value_array.append('error')

            self.dataframe = pd.DataFrame(index=company_id_array)
            self.dataframe['assetName'] = company_name_array
            self.dataframe['assetCurrency'] = asset_currency_array
            self.dataframe['assetType'] = asset_type_array
            self.dataframe['assetValue'] = asset_value_array
            self.dataframe['assetValue'] = pd.to_numeric(self.dataframe['assetValue'],errors='coerce')

            resp = r.putRatio([13, 12, 10, 9], company_id_array, None, START_DATE, END_DATE, 'yearly')

            self.dataframe['ROI'] = None
            self.dataframe['annualROI'] = None
            self.dataframe['sharpe'] = None
            self.dataframe['stdDev'] = None

            for i in company_id_array:
                if i == 2201:
                    print((resp[str(i)]['12']['value']).replace(',', '.'))
                self.dataframe.loc[i, 'ROI'] = (resp[str(i)]['13']['value']).replace(',', '.')
                self.dataframe.loc[i, 'annualROI'] = (resp[str(i)]['9']['value']).replace(',', '.')
                self.dataframe.loc[i, 'sharpe'] = (resp[str(i)]['12']['value']).replace(',', '.')
                self.dataframe.loc[i, 'stdDev'] = (resp[str(i)]['10']['value']).replace(',', '.')
                self.dataframe.loc[i, 'ROIType'] = (resp[str(i)]['13']['type'])

            self.dataframe['ROI'] = pd.to_numeric(self.dataframe['ROI'],errors='coerce')
            self.dataframe['annualROI'] = pd.to_numeric(self.dataframe['annualROI'],errors='coerce')
            self.dataframe['sharpe'] = pd.to_numeric(self.dataframe['sharpe'],errors='coerce')
            self.dataframe['stdDev'] = pd.to_numeric(self.dataframe['stdDev'],errors='coerce')
            
            self.dataframe['quantity'] = 0
            self.dataframe['totalValue'] = 0
            self.dataframe['NAVPercentage'] = 0
        else:
            self.dataframe = dataframe.copy()
        
        self.cov = pd.DataFrame(index=self.dataframe.index)
        for i in self.dataframe.index:
            self.cov[i] = None

    def get_dataframe(self):
        return self.dataframe
        
    def get_index(self):
        return self.dataframe.index.tolist()
    
    def is_fund(self, id):
        return 'FUND' in assetDF.loc[id, 'assetType']
    
    def get_nb_asset(self):
        return len(self.dataframe[self.dataframe['quantity'] != 0])
    
    def update_nav(self):
        self.dataframe['NAVPercentage'] = self.dataframe['totalValue'] / self.dataframe['totalValue'].sum()
        
    def get_asset(self, id, n):
        quantity = self.dataframe.loc[id, 'quantity']
        self.dataframe.loc[id, 'quantity'] += n if n + quantity >= 0 or n > 0 else 0
        self.dataframe.loc[id, 'totalValue'] = self.dataframe.loc[id, 'assetValue'] * self.dataframe.loc[id, 'quantity']
        self.update_nav()
        return self.dataframe
    
    def print_cov(self):
        return self.cov
    
    # Fills in correlation matrix but takes 3 minutes
    def init_correlation(self):
        for nb, i in enumerate(self.dataframe.index):
            correlationResp = r.putRatio([11], b.get_index(), i, START_DATE, END_DATE, 'yearly')
            for j in self.dataframe.index:
                if correlationResp[str(j)]['11']['type'] == 'double' and cov.loc[i, j] is None:
                    self.cov.loc[i, j] = float(correlationResp[str(j)]['11']['value'].replace(',', '.'))
                    self.cov.loc[j, i] = self.cov.loc[i, j]
                else:
                    self.cov.loc[i, j] = float('nan')
            print("{} / {}".format(nb, len(self.dataframe.index)))
        return self.cov
    
    def get_covariance_unused(self, i, j):
        return self.cov.loc[i, j]
    
    def get_covariance(self, i, j):
        if self.cov.loc[i, j] is None or math.isnan(self.cov.loc[i, j]):
            correlationResp = r.putRatio([11], [i], j, START_DATE, END_DATE, "yearly")
            if correlationResp[str(i)]['11']['type'] == 'double':
                correlation = float((correlationResp[str(i)]['11']['value']).replace(',', '.'))
                self.cov.loc[i, j] = correlation
                self.cov.loc[j, i] = correlation
            else:
                correlation = float('NaN')
        else:
            correlation = self.cov.loc[i, j]
        stdDev_i = self.dataframe.loc[i, 'stdDev']
        stdDev_j = self.dataframe.loc[j, 'stdDev']
        print((stdDev_i, stdDev_j))
        print(correlation)
        return correlation * stdDev_i * stdDev_j
    
    # Compute sharpe of portfolio
    def get_variance(self):
        sum = 0
        for i in self.dataframe[self.dataframe['NAVPercentage'] != 0].index:
            for j in self.dataframe[self.dataframe['NAVPercentage'] != 0].index:
                wi = self.dataframe.loc[i, 'NAVPercentage']
                wj = self.dataframe.loc[j, 'NAVPercentage']
                cov = self.get_covariance(i, j)
                print((wi, wj))
                print(cov)
                sum += wi * wj * cov
        return sum

    def get_rendement(self):
        return (self.dataframe['NAVPercentage'] * self.dataframe['annualROI']).sum()

    def get_sharpe(self):
        rendement = self.get_rendement()
        variance = self.get_variance()
        print((rendement, np.sqrt(variance)))
        if variance == 0:
            return 'error'
        return (rendement - 0.0005) / np.sqrt(variance)

In [169]:
#test = AssetDataframe()
b = Portfolio()
test = b.get_dataframe().sort_values(by=['sharpe'], ascending=False)
test[test.assetType == 'PORTFOLIO']

0.025232509524


,assetName,assetCurrency,assetType,assetValue,ROI,annualROI,sharpe,stdDev,ROIType,quantity,totalValue,NAVPercentage
2201,REF,EUR,PORTFOLIO,793.536,0.048694,0.011023,0.025233,0.238706,percent,0,0,0
1824,EPITA_PTF_5,EUR,PORTFOLIO,NaN,NaN,NaN,NaN,NaN,error,0,0,0


In [170]:
b.get_asset(1900, 0).sort_values(by=['NAVPercentage'], ascending=False)
b.get_asset(1529, 6).sort_values(by=['NAVPercentage'], ascending=False)

,assetName,assetCurrency,assetType,assetValue,ROI,annualROI,sharpe,stdDev,ROIType,quantity,totalValue,NAVPercentage
1529,ESSILORLUXOTTICA,EUR,STOCK,118.250,-0.347621,-0.093791,-0.780142,0.126632,percent,6,709.5,1.0
1846,1000MERCIS,EUR,STOCK,40.900,-0.391059,-0.108075,-0.500443,0.225949,percent,0,0.0,0.0
2040,FFP,EUR,STOCK,68.250,-0.190458,-0.047550,-0.165243,0.318015,percent,0,0.0,0.0
2039,FF GROUP,EUR,STOCK,19.200,-0.654584,-0.217376,-0.574321,0.387197,percent,0,0.0,0.0
2038,FERRAGAMO SPA,EUR,STOCK,13.540,0.130020,0.028585,0.205798,0.114604,percent,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1929,BUSINESS & DECISIONS,EUR,STOCK,4.650,0.000335,0.000077,-0.018121,0.271664,percent,0,0.0,0.0
1928,BURELLE SA,EUR,STOCK,819.490,-0.232855,-0.059290,-0.195595,0.328691,percent,0,0.0,0.0
1468,BUREAU VERITAS - S,EUR,STOCK,19.325,-0.175350,-0.043480,-0.259622,0.186734,percent,0,0.0,0.0
1927,BREMBO SPA,EUR,STOCK,52.000,-0.783651,-0.297407,-0.684020,0.442103,percent,0,0.0,0.0


In [171]:
r.getPortfolio(2201)

{'label': 'REF',
 'currency': {'code': 'EUR'},
 'type': 'front',
 'values': {'2016-01-01': [{'asset': {'asset': 1860, 'quantity': 6.0}},
   {'asset': {'asset': 1900, 'quantity': 10.0}},
   {'currency': {'currency': {'code': 'EUR'}, 'amount': 300.0}}]}}

In [172]:
company_id_array = [int(asset['ASSET_DATABASE_ID']['value']) for asset in pars]
resp = r.putRatio([13, 12, 10, 9], [2201], None, START_DATE, END_DATE, 'yearly')
(resp['2201']['12']['value']).replace(',', '.')

'0.025232509524'

In [173]:
b.get_sharpe()

(0.126632087171, 0.126632087171)
0.8
(1.0, 1.0)
0.012828548401026993
(-0.093790994702, 0.11326318201881401)


-0.8324946643856204

In [129]:
b.get_sharpe()

(0.08760455164364911, 0.08760455164364911)
0.5152313178806063
(0.08760455164364911, 0.9123954483563509)
0.13404650349591649
(0.9123954483563509, 0.08760455164364911)
0.1340465034959165
(0.9123954483563509, 0.9123954483563509)
0.09253332788690811
(0.019638048633876217, 0.3200213175278124)


0.05980241810676556

In [149]:
r.putRatio([11], [1900], 1860, START_DATE, END_DATE, 'yearly')

{'1900': {'11': {'type': 'double', 'value': '0,491128639063'}}}

In [131]:
b.print_cov()

,1845,1846,2122,2123,2124,1428,1847,1848,1849,2154,...,1793,1794,1940,1906,2097,1879,1997,1991,2106,1984
1845,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1846,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2122,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2123,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2124,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1879,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1997,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1991,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2106,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [475]:
def init_asset_dataframe(pars):
  company_id_array = []
  company_name_array = []
  asset_currency_array = []
  asset_type_array = []
  asset_value_array = []

  for asset in pars:
    company_id_array.append(int(asset['ASSET_DATABASE_ID']['value']))
    company_name_array.append(asset['LABEL']['value'])
    asset_currency_array.append(asset['CURRENCY']['value'])
    asset_type_array.append(asset['TYPE']['value'])
    asset_value_array.append(asset['LAST_CLOSE_VALUE_IN_CURR']['value'].split()[0].replace(',', '.')) if 'LAST_CLOSE_VALUE_IN_CURR' in asset else asset_value_array.append('error')
  
  assetDF = pd.DataFrame(index=company_id_array)
  assetDF['assetName'] = company_name_array
  assetDF['assetCurrency'] = asset_currency_array
  assetDF['assetType'] = asset_type_array
  assetDF['assetValue'] = asset_value_array
  assetDF['assetValue'] = pd.to_numeric(assetDF['assetValue'],errors='coerce')

  return assetDF

In [476]:
assetDF = init_asset_dataframe(pars)
assetDF[pd.isnull(assetDF['assetName'])]
#assetDF

,assetName,assetCurrency,assetType,assetValue


In [480]:
def fill_asset_dataframe(dataframe):
    company_id_array = [int(asset['ASSET_DATABASE_ID']['value']) for asset in pars]
    resp = r.putRatio([13, 12, 10, 9], company_id_array, None, START_DATE, END_DATE, None)

    '''
    # Try to get values for asset not found before
    for id in assetDF[pd.isnull(assetDF['assetValue'])].index:
    assetResp = r.getAsset(id, START_DATE)
    if 'LAST_CLOSE_VALUE_IN_CURR' in assetResp:
      dataframe.loc[id, 'assetValue'] = assetResp['LAST_CLOSE_VALUE_IN_CURR']['value'].split()[0].replace(',', '.')
    '''
    print(resp['1458'])
    dataframe['ROI'] = None
    dataframe['annualROI'] = None
    dataframe['sharpe'] = None
    dataframe['stdDev'] = None
    
    for id in company_id_array:
        dataframe.loc[id, 'ROI'] = (resp[str(id)]['13']['value']).replace(',', '.')
        dataframe.loc[id, 'annualROI'] = (resp[str(id)]['9']['value']).replace(',', '.')
        dataframe.loc[id, 'sharpe'] = (resp[str(id)]['12']['value']).replace(',', '.')
        dataframe.loc[id, 'stdDev'] = (resp[str(id)]['10']['value']).replace(',', '.')
        dataframe.loc[id, 'ROIType'] = (resp[str(id)]['13']['type'])

    dataframe['ROI'] = pd.to_numeric(dataframe['ROI'],errors='coerce')
    dataframe['annualROI'] = pd.to_numeric(dataframe['annualROI'],errors='coerce')
    dataframe['sharpe'] = pd.to_numeric(dataframe['sharpe'],errors='coerce')
    dataframe['stdDev'] = pd.to_numeric(dataframe['stdDev'],errors='coerce')
    #dataframe[dataframe.ROIType == 'error']
    return dataframe

In [482]:
assetDF = fill_asset_dataframe(assetDF)
assetDF.sort_values(by=['sharpe', 'ROI'], ascending=False, inplace=True)
assetDF

{'13': {'type': 'percent', 'value': '0,612105757127'}, '9': {'type': 'percent', 'value': '0,116399757432'}, '12': {'type': 'double', 'value': '0,417921798934'}, '10': {'type': 'percent', 'value': '0,266556465148'}}


,assetName,assetCurrency,assetType,assetValue,ROI,annualROI,sharpe,stdDev,ROIType
2024,EUROFINS-CEREP REGRP,EUR,STOCK,100.0000,56.979961,1.550161,2.762251,0.559385,percent
1958,CHRISTIAN DIOR,EUR,STOCK,145.9500,3.753575,0.432531,2.173915,0.196664,percent
2165,CAC 40 GR,EUR,INDEX,10910.5000,1.642355,0.251125,1.576531,0.156118,percent
2154,ABERDEEN CHINA EQUITY A2,USD,FUND,20.9896,5.448789,0.536899,1.522731,0.349306,percent
1971,CRCAM DU LANGUED,EUR,STOCK,57.0000,1.682203,0.255450,1.491983,0.167864,percent
...,...,...,...,...,...,...,...,...,...
1572,HENKEL AND KGAA PRF,EUR,STOCK,104.3720,-0.623559,-0.201699,-1.052137,0.196457,percent
1992,DEXIA SA,EUR,STOCK,18.2000,-0.814100,-0.321556,-1.064825,0.306676,percent
2142,AMIRAL - SEXTANT GRAND LARGE - A (EUR),EUR,FUND,403.5600,-0.898064,-0.409329,-1.188230,0.348695,percent
2123,29 HAUSSMANN CROISSANCE EUROPE D,EUR,FUND,1670.6500,-0.973426,-0.566767,-1.236339,0.462468,percent


In [15]:
# Lines where no value is given
assetDF[pd.isnull(assetDF['assetValue'])]

,assetName,assetCurrency,assetType,assetValue,ROI,annualROI,sharpe,stdDev,ROIType
2087,DELL INC,USD,STOCK,NaN,2.265339,0.313708,1.043443,0.295855,percent
2048,FRESENIUS MEDICA,EUR,STOCK,NaN,2.138848,0.301795,1.025301,0.289471,percent
2130,ACA - HORIZON MONDE (EUR),EUR,FUND,NaN,0.651955,0.122703,0.761507,0.154566,percent
1855,ADOMOS,EUR,STOCK,NaN,0.810565,0.146689,0.684980,0.206850,percent
1867,ALLIANZ SE-VINK,EUR,STOCK,NaN,0.819439,0.147982,0.609606,0.234548,percent
1824,EPITA_PTF_5,EUR,PORTFOLIO,NaN,0.304873,0.063279,0.419509,0.138921,percent
2125,ABERDEEN GLOBAL ASIA PACIFIC EQUITY E2,EUR,FUND,NaN,0.358237,0.073151,0.369883,0.184250,percent
2155,ABERDEEN FUNDS GLOBAL PHARMA EQUITY - A,USD,FUND,NaN,0.353171,0.072227,0.367198,0.183080,percent
1955,CGGVERITAS,EUR,STOCK,NaN,0.161844,0.035193,0.164434,0.183619,percent
1488,CPR-SILVER AGE-I,EUR,FUND,NaN,0.017295,0.003961,-0.004597,0.225923,percent


In [16]:
# Try to fill value at day - 1
def try_filling_value():
  for id in assetDF[pd.isnull(assetDF['assetValue'])].index:
    day = 20
    assetResp = r.getAsset(id, datetime(2016, 5, day).isoformat('T'))
    if 'LAST_CLOSE_VALUE_IN_CURR' in assetResp:
      dataframe.loc[id, 'assetValue'] = assetResp['LAST_CLOSE_VALUE_IN_CURR']['value'].split()[0].replace(',', '.')

In [17]:
len(assetDF[pd.isnull(assetDF['assetValue'])])

17

In [18]:
def is_fund(id):
  return 'FUND' in assetDF.loc[id, 'assetType']

In [19]:
is_fund(2159)

True

### Portfolio creation

Let's then create our own portfolio dataframe

In [20]:
def init_portfolio(dataframe):
  portfolioDF = dataframe.copy()
  portfolioDF['quantity'] = 0
  portfolioDF['totalValue'] = 0
  portfolioDF['NAVPercentage'] = 0
  return portfolioDF

In [21]:
portfolioDF = init_portfolio(assetDF)
portfolioDF

,assetName,assetCurrency,assetType,assetValue,ROI,annualROI,sharpe,stdDev,ROIType,quantity,totalValue,NAVPercentage
1458,BBVA,EUR,STOCK,5.8410,56.979961,1.550161,2.762251,0.559385,percent,0,0,0
1514,EIFFAGE,EUR,STOCK,66.8100,3.753575,0.432531,2.173915,0.196664,percent,0,0,0
1914,BELGACOM SA,EUR,STOCK,29.4700,1.642355,0.251125,1.576531,0.156118,percent,0,0,0
1901,AVENIR TELECOM,EUR,STOCK,0.0600,5.448789,0.536899,1.522731,0.349306,percent,0,0,0
2092,EXXON MOBIL,USD,STOCK,89.2400,1.682203,0.255450,1.491983,0.167864,percent,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1787,UNI EQUIT.WORLD EUR CL.B2 C.3D,EUR,FUND,1994.9000,-0.623559,-0.201699,-1.052137,0.196457,percent,0,0,0
1596,Lazard Dividendes Min VaR C,EUR,FUND,266.0200,-0.814100,-0.321556,-1.064825,0.306676,percent,0,0,0
1883,AQUILA,EUR,STOCK,5.9900,-0.898064,-0.409329,-1.188230,0.348695,percent,0,0,0
2159,AMUNDI ETF MSCI,USD,ETF FUND,10.6612,-0.973426,-0.566767,-1.236339,0.462468,percent,0,0,0


In [22]:
resp = r.getAsset(1514, START_DATE)
resp

{'LABEL': {'type': 'string', 'value': 'EIFFAGE'},
 'TYPE': {'type': 'asset_type', 'value': 'STOCK'},
 'CURRENCY': {'type': 'asset_currency', 'value': 'EUR'},
 'ASSET_DATABASE_ID': {'type': 'int32', 'value': '1514'},
 'LAST_CLOSE_VALUE_IN_CURR': {'type': 'currency_value', 'value': '66,81 EUR'}}

In [23]:
def update_nav(portfolio):
  portfolio['NAVPercentage'] = portfolio['totalValue'] / portfolio['totalValue'].sum()

In [24]:
# Allows buying or selling an asset with id
def get_asset(portfolio, id, n):
  quantity = portfolio.loc[id, 'quantity']
  portfolio.loc[id, 'quantity'] += n if n + quantity >= 0 or n > 0 else 0
  portfolio.loc[id, 'totalValue'] = portfolio.loc[id, 'assetValue'] * portfolio.loc[id, 'quantity']
  update_nav(portfolio)
  return portfolio

In [25]:
def get_nb_asset(portfolio):
  return len(portfolio[portfolio['quantity'] != 0])

In [26]:
def get_best_asset(portfolio, mode):
  if get_nb_asset(portfolio) < 15:
    if mode == 'STOCK':
      portfolio = portfolio[portfolio['assetType'] == 'STOCK']
    elif mode != 'STOCK' and mode != 'ALL':
      portfolio = portfolio[portfolio['assetType'] != 'STOCK']
    for id in portfolio.index:
      if portfolio.loc[id, 'NAVPercentage'] < 0.01:
        while portfolio.loc[id, 'NAVPercentage'] < 0.1:
          get_asset(portfolio, id, START_DATE, 1)
        break
  return portfolio

In [27]:
portfolioDF = get_asset(portfolioDF, 1514, 2)

In [28]:
#portfolioDF = get_best_asset(portfolioDF, 'ALL')
portfolioDF.sort_values(by=['NAVPercentage'], ascending=False)

,assetName,assetCurrency,assetType,assetValue,ROI,annualROI,sharpe,stdDev,ROIType,quantity,totalValue,NAVPercentage
1514,EIFFAGE,EUR,STOCK,66.8100,3.753575,0.432531,2.173915,0.196664,percent,2,133.62,1.0
1458,BBVA,EUR,STOCK,5.8410,56.979961,1.550161,2.762251,0.559385,percent,0,0.00,0.0
1792,VINCI,EUR,STOCK,67.2000,-0.084677,-0.020194,-0.175586,0.143486,percent,0,0.00,0.0
1457,BAYER AG-REG,EUR,STOCK,86.2500,-0.162106,-0.039960,-0.168210,0.267286,percent,0,0.00,0.0
1455,Barclays Euro Govt Inflation-Linked Bond Index...,EUR,INDEX,207.8996,-0.105003,-0.025254,-0.167527,0.180595,percent,0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1578,ILIAD SA,EUR,STOCK,194.4500,0.623374,0.118194,0.315541,0.358730,percent,0,0.00,0.0
1579,INDUSTRIA DE DESINO TEXTIL,EUR,STOCK,30.2100,0.328023,0.067599,0.327346,0.191231,percent,0,0.00,0.0
2036,FAURECIA,EUR,STOCK,34.7950,0.500962,0.098162,0.327773,0.284227,percent,0,0.00,0.0
2140,Altitude Profil Midcaps,EUR,FUND,137.5400,0.293060,0.061051,0.340400,0.164663,percent,0,0.00,0.0


In [50]:
def fill_covariance():
  cov = pd.DataFrame(index=assetDF.index)
  for i in assetDF.index:
    for j in assetDF.index:
      cov[i, j] = get_covariance(i, j)
  return cov
  
def get_covariance(i, j):
  correlationResp = r.putRatio([11], [i], j, START_DATE, END_DATE, None)
  correlation = (correlationResp[str(i)]['11']['value']).replace(',', '.')
  stdDev_i = portfolioDF.loc[i, 'stdDev']
  stdDev_j = portfolioDF.loc[j, 'stdDev']
  return float(correlation) * stdDev_i * stdDev_j

# Compute sharpe of portfolio
def get_variance(portfolio):
  sum = 0
  for i in portfolio[portfolio['quantity'] != 0].index:
    for j in portfolio[portfolio['quantity'] != 0].index:
      sum += portfolio.loc[i, 'NAVPercentage'] * portfolio.loc[j, 'NAVPercentage'] * get_covariance(i, j)
  return sum

def get_rendement(portfolio):
  return (portfolio['NAVPercentage'] * portfolio['ROI']).sum()

def get_sharpe(portfolio):
  rendement = get_rendement(portfolio)
  variance = get_variance(portfolio)
  return (rendement - 0.05) / np.sqrt(variance)

In [ ]:
get_rendement(portfolioDF)

3.753575437399

In [ ]:
get_sharpe(portfolioDF)

18.840440473339367

In [ ]:
portfolioDF[portfolioDF.assetType == 'PORTFOLIO']

,assetName,assetCurrency,assetType,assetValue,ROI,annualROI,sharpe,stdDev,ROIType,quantity,totalValue,NAVPercentage
1824,EPITA_PTF_5,EUR,PORTFOLIO,NaN,0.304873,0.063279,0.419509,0.138921,percent,0,0.0,0.0
2201,REF,EUR,PORTFOLIO,NaN,0.188414,0.040604,0.315332,0.112911,percent,0,0.0,0.0


In [420]:
#r.putPortfolio(1824, 'EPITA_PTF_5', EUR, 'front', )
r.getPortfolio(2201)

{'label': 'REF',
 'currency': {'code': 'EUR'},
 'type': 'front',
 'values': {'2016-01-01': [{'asset': {'asset': 1860, 'quantity': 6.0}},
   {'asset': {'asset': 1900, 'quantity': 10.0}},
   {'currency': {'currency': {'code': 'EUR'}, 'amount': 300.0}}]}}

In [423]:
resp = r.putRatio([12], [2201], None, START_DATE, END_DATE, None)

res = [(resp[id]['12']['value']).replace(',', '.') for id in resp]
res

['0.146571943186']

In [ ]:
portfolioDF.NAVPercentage.values

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
from string import ascii_uppercase as ASCIIUP
import time
import random
import numpy as np

def fake_sharpefunction(elms):
  return -np.std(elms)

class TreeCombi:
    def __init__(self, elems, lr=0.01):
        # Data
        self.elems = elems
        self.end = len(elems)
        # Iter
        self.nb_combis = pow(2, self.end) - 1
        self.iter = 0
        self.lr = lr
        # Time
        self.t0 = time.time()
        self.t1 = time.time()
        self.elp = 0
        self.est = 0
        print(f'nb_combis: {self.nb_combis}')
    
    def __process(self, index):
        portSharp = fake_sharpefunction(self.elems) # FIXME: compute sharpe for port
        portSharp_1 = portSharp
        while True:
          # save navPer and portSharp
          navPer = self.elems[index]
          portSharp = portSharp_1
          # update df navPer according to learning rate self.lr
          self.elems[index] += self.lr
          # compute new portShrap
          portSharp_1 = fake_sharpefunction(self.elems)
          if portSharp_1 <= portSharp:
            break
        self.elems[index] = navPer
        print(portSharp)

    def __call__(self, start=-1, s=''):
        if start != -1:  # Process element at start index
            s += str(self.elems[start]) + "," # Just for testing purpose
            self.iter += 1
            ti = time.time()  # took: Current iteration time

            # Processing
            #time.sleep(0.1 + random.random() / 2)  # Process element...
            self.__process(start)
            print(s, " - ", self.elems)

            # Time & ouptut
            self.t1 = time.time()
            self.elp = self.t1 - self.t0  # cur_tm: total elapsed time
            self.est = (self.elp / self.iter) * self.nb_combis  # est_tm: total estimated time
            print(f"iter {self.iter}/{self.nb_combis}: took:{self.t1 - ti:.1f}, cur_tm:{self.elp:.1f}, est_tm:{self.est:.1f}, left_tm:{self.est - self.elp:.1f}")
        else:
            self.t0 = time.time()
        for i in range(start + 1, self.end):
            self(i, s)

X = np.array(list(range(4))).astype("float")
print(X)
#t = TreeCombi(list(ASCIIUP)[:4])
t = TreeCombi(X)
t()

[0. 1. 2. 3.]
nb_combis: 15
-0.7071067811865476
0.0,  -  [2. 1. 2. 3.]
iter 1/15: took:0.0, cur_tm:0.0, est_tm:0.1, left_tm:0.1
-0.4082508420077046
0.0,1.0,  -  [2.   2.33 2.   3.  ]
iter 2/15: took:0.0, cur_tm:0.0, est_tm:0.1, left_tm:0.1
-0.36030369134939494
0.0,1.0,2.0,  -  [2.   2.33 2.44 3.  ]
iter 3/15: took:0.0, cur_tm:0.0, est_tm:0.1, left_tm:0.0
-0.36030369134939494
0.0,1.0,2.0,3.0,  -  [2.   2.33 2.44 3.  ]
iter 4/15: took:0.0, cur_tm:0.0, est_tm:0.0, left_tm:0.0
-0.36030369134939494
0.0,1.0,3.0,  -  [2.   2.33 2.44 3.  ]
iter 5/15: took:0.0, cur_tm:0.0, est_tm:0.0, left_tm:0.0
-0.36030369134939494
0.0,2.4399999999999906,  -  [2.   2.33 2.44 3.  ]
iter 6/15: took:0.0, cur_tm:0.0, est_tm:0.0, left_tm:0.0
-0.36030369134939494
0.0,2.4399999999999906,3.0,  -  [2.   2.33 2.44 3.  ]
iter 7/15: took:0.0, cur_tm:0.0, est_tm:0.0, left_tm:0.0
-0.36030369134939494
0.0,3.0,  -  [2.   2.33 2.44 3.  ]
iter 8/15: took:0.0, cur_tm:0.0, est_tm:0.0, left_tm:0.0
-0.35440090293338683
2.329999999